In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Company ABC - Pincode Zones.csv
/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Courier Company - Rates.csv
/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Company ABC - SKU Master.csv
/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Company ABC- Order Report.csv
/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Courier Company - Invoice.csv


In [2]:
#Load all datasets

pincode_zones=pd.read_csv('/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Company ABC - Pincode Zones.csv')
company_rates=pd.read_csv('/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Courier Company - Rates.csv')
sku_master=pd.read_csv('/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Company ABC - SKU Master.csv')
order_report=pd.read_csv('/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Company ABC- Order Report.csv')
invoice=pd.read_csv('/kaggle/input/b2b-ecommerce-dataset-charges-accuracy-analysis/Courier Company - Invoice.csv')

In [3]:
print("Order Report:")
print(order_report.head())
print("\nSKU Master:")
print(sku_master.head())
print("\nPincode Mapping:")
print(pincode_zones.head())
print("\nCourier Invoice:")
print(invoice.head())
print("\nCourier Company rates:")
print(company_rates.head())


Order Report:
   ExternOrderNo          SKU  Order Qty
0     2001827036  8.90422E+12        1.0
1     2001827036  8.90422E+12        1.0
2     2001827036  8.90422E+12        1.0
3     2001827036  8.90422E+12        1.0
4     2001827036  8.90422E+12        1.0

SKU Master:
           SKU  Weight (g)
0  8.90422E+12         210
1  8.90422E+12         165
2  8.90422E+12         113
3  8.90422E+12          65
4  8.90422E+12         120

Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone
0             121003            507101    d
1             121003            486886    d
2             121003            532484    d
3             121003            143001    b
4             121003            515591    d

Courier Invoice:
        AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            2.50             121003 

In [4]:
# Check for missing values
print("\nMissing values in Website Order Report:")
print(order_report.isnull().sum())
print("\nMissing values in SKU Master:")
print(sku_master.isnull().sum())
print("\nMissing values in Pincode Mapping:")
print(pincode_zones.isnull().sum())
print("\nMissing values in Courier Invoice:")
print(invoice.isnull().sum())
print("\nMissing values in courier company rates:")
print(company_rates.isnull().sum())


Missing values in Website Order Report:
ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64

Missing values in SKU Master:
SKU           0
Weight (g)    0
dtype: int64

Missing values in Pincode Mapping:
Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

Missing values in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in courier company rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed         0
rto_a_additional    0
rto_b_fixed         0
rto_b_additional    0
rto_c_fixed         0
rto_c_additional    0
rto_d_fixed         0
rto_d_additional    0


SKU is a common field between order report and SKU master , lets merge this data for easy analysis

In [5]:
new_order_report = pd.merge(order_report, sku_master, on='SKU')
print(new_order_report.head())

   ExternOrderNo          SKU  Order Qty  Weight (g)
0     2001827036  8.90422E+12        1.0         210
1     2001827036  8.90422E+12        1.0         165
2     2001827036  8.90422E+12        1.0         113
3     2001827036  8.90422E+12        1.0          65
4     2001827036  8.90422E+12        1.0         120


ExternOrderNo is same as OrderID , let's rename the column

In [6]:
new_order_report = new_order_report.rename(columns={'ExternOrderNo': 'Order ID'})

We can do a pincode mapping with the invoice dataset.

To do , lets extract the unique pincodes from pincode_zones

In [7]:
unique_pincodes = pincode_zones.drop_duplicates(subset=['Customer Pincode'])


Let's extract only the needed columns from invoice and merge it with above dataframe

In [8]:
invoice_updated= invoice[['Order ID', 'Customer Pincode','Type of Shipment']]


Now let's merge above two dataframes to form a new dataframe as 'pincode_mapping'

In [9]:
pincode_mapping=invoice_updated.merge(unique_pincodes,on='Customer Pincode')

In [10]:
pincode_mapping.head()

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001806232,507101,Forward charges,121003,d
1,2001806273,486886,Forward charges,121003,d
2,2001806408,532484,Forward charges,121003,d
3,2001806458,143001,Forward charges,121003,b
4,2001807012,515591,Forward charges,121003,d


Now we can merge this with the new_order_report dataframe

In [11]:
updated_order_report=new_order_report.merge(pincode_mapping,on='Order ID')

Let's see the final dataframes we are going to use for further analysis

In [12]:
print('OrderReport:')
print(updated_order_report.head())

print('/n Invoice:')
print(invoice.head())

print('/n Company Rates:')
print(company_rates.head())


OrderReport:
     Order ID          SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8.90422E+12        1.0         210            173213   
1  2001827036  8.90422E+12        1.0         165            173213   
2  2001827036  8.90422E+12        1.0         113            173213   
3  2001827036  8.90422E+12        1.0          65            173213   
4  2001827036  8.90422E+12        1.0         120            173213   

  Type of Shipment  Warehouse Pincode Zone  
0  Forward charges             121003    e  
1  Forward charges             121003    e  
2  Forward charges             121003    e  
3  Forward charges             121003    e  
4  Forward charges             121003    e  
/n Invoice:
        AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            2.50             121003   
3  1091117

Now let's change weights from gram to kg for a neat analysis and apply weight slabs to any weights column in the above dataframes

In [13]:
updated_order_report['Weights (Kgs)'] = updated_order_report['Weight (g)'] / 1000

Below function first calculates the remainder of the weight divided by 1 and rounds it to one decimal place. 

If the remainder is 0.0, it means the weight is a multiple of 1 KG, and the function returns the weight as it is.

If the remainder is greater than 0.5, it means that the weight exceeds the next half KG slab. 

In this case, the function rounds the weight to the nearest integer and adds 1.0 to it, which represents the next heavier slab.

If the remainder is less than or equal to 0.5, it means the weight falls into the current half-KG bracket. 

In this case, the function rounds the weight to the nearest integer and adds 0.5 to it, which represents the current weight slab.

In [14]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

updated_order_report['Weight Slab (KG)'] = updated_order_report['Weights (Kgs)'].apply(weight_slab)
invoice['Weight Slab Charged']=(invoice['Charged Weight']).apply(weight_slab)

Let's rename columns for easy understanding

In [15]:
invoice = invoice.rename(columns={'Zone': 'Delivery Zone Allocated by Courier Company'})
updated_order_report = updated_order_report.rename(columns={'Zone': 'Delivery Zone As Per Company'})
updated_order_report = updated_order_report.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per Company'})

Now let's calculate expected charges and then find difference in the charges made 

In [16]:
total_expected_charge = []

#loop through each row of the ‘updated_order_report’ DataFrame to calculate the expected charges based on company’s tariffs

for _, row in updated_order_report.iterrows():
    
    fwd_category = 'fwd_' + row['Delivery Zone As Per Company']
    fwd_fixed = company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per Company']
    rto_fixed = company_rates.at[0, rto_category + '_fixed']
    rto_additional = company_rates.at[0, rto_category + '_additional']

    #determine the weight of the slab for each row
    
    weight_slab = row['Weight Slab As Per Company']
    
    #If the shipment type is ‘Forward Charges’, we calculate the additional weight beyond the basic weight slab (0.5 KG) and apply the corresponding additional charges.
    #For “Forward and RTO Charges” shipments, we consider additional charges for term and RTO components.

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

updated_order_report['Expected Charge as per company'] = total_expected_charge
print(updated_order_report.head())

     Order ID          SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8.90422E+12        1.0         210            173213   
1  2001827036  8.90422E+12        1.0         165            173213   
2  2001827036  8.90422E+12        1.0         113            173213   
3  2001827036  8.90422E+12        1.0          65            173213   
4  2001827036  8.90422E+12        1.0         120            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per Company  \
0  Forward charges             121003                            e   
1  Forward charges             121003                            e   
2  Forward charges             121003                            e   
3  Forward charges             121003                            e   
4  Forward charges             121003                            e   

   Weights (Kgs)  Weight Slab As Per Company  Expected Charge as per company  
0          0.210                         0.5                            5

Let's merge the datasets to display final output

In [17]:
output = updated_order_report.merge(invoice, on='Order ID')
print(output.head())

     Order ID          SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8.90422E+12        1.0         210              173213   
1  2001827036  8.90422E+12        1.0         165              173213   
2  2001827036  8.90422E+12        1.0         113              173213   
3  2001827036  8.90422E+12        1.0          65              173213   
4  2001827036  8.90422E+12        1.0         120              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per Company  \
0    Forward charges               121003                            e   
1    Forward charges               121003                            e   
2    Forward charges               121003                            e   
3    Forward charges               121003                            e   
4    Forward charges               121003                            e   

   Weights (Kgs)  Weight Slab As Per Company  Expected Charge as per company  \
0          0.210                    

Now let's calculate the difference in charges

In [18]:
output['Difference (Rs.)'] = output['Billing Amount (Rs.)'] - output['Expected Charge as per company']

df_new = output[['Order ID', 'Difference (Rs.)', 'Expected Charge as per company']]

print(df_new.head())

     Order ID  Difference (Rs.)  Expected Charge as per company
0  2001827036              61.3                            56.6
1  2001827036              61.3                            56.6
2  2001827036              61.3                            56.6
3  2001827036              61.3                            56.6
4  2001827036              61.3                            56.6


Now lets find the number of orders correctly charged or under charged or over charged

In [19]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per company'].sum()

# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

                                         Description  Count  Amount (Rs.)
0  Total Orders where ABC has been correctly charged    741       32427.3
1        Total Orders where ABC has been overcharged  22339     1947835.8
2       Total Orders where ABC has been undercharged    508      -13369.4


In [20]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            hole=0.4))
fig.update_layout(title='Proportion')

fig.show()